## Method 3 (Subspace alignment based Domain adaptation)

In [ ]:
import torchvision.ops.boxes as bops
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from torch import nn

Papers 

 1. https://arxiv.org/pdf/1507.05578.pdf

 2.  https://openaccess.thecvf.com/content_iccv_2013/papers/Fernando_Unsupervised_Visual_Domain_2013_ICCV_paper.pdf

In [ ]:
# Initialize model and load (deep) fine-tuned weights

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = 2
model = get_model_from_pretrained(num_classes)
model.to(device)

model.load_state_dict(torch.load('your_path/model_25.pt'))


### Source data

In [ ]:
# Source data/distribution = training set
train_valid_img,_= get_img_with_bbox(train_ann_path)  
training_data= CustomImageDataset(train_ann_path,img_folder, train_valid_img)
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True,collate_fn=utils.collate_fn)

**Construct source matrix:** 

We keep output of model.roi_heads.box_head (vector of size 1024) as feature representations of bounding boxes extracted by the RPN (region proposal network). For us to stack a box representation to the source matrix, it has to have a IoU > thres_IoU with the ground truth of the given image. 


In [ ]:

thres_IoU= 0.50
count=0

X_source=torch.tensor([])
bbox_idx=torch.arange(1000)

model.eval()

for images, targets in train_dataloader: 
    images = [image.to(device) for image in images]
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    count+=1

    if count%100==0:
      print(count)

    with torch.no_grad():
        outputs = []
        hook = model.rpn.register_forward_hook(
        lambda self, input, output: outputs.append(output))

        outputs1 = []
        hook1 = model.roi_heads.box_head.register_forward_hook(
        lambda self, input, output: outputs1.append(output))

        res = model(images)
        hook.remove()
        hook1.remove()

    coords = outputs[0][0][0].cpu() # [1000,4]
    feat=outputs1[0].cpu() # [1000, 1024]

    gt = targets[0]['boxes'].cpu()

    bbox_idx_to_keep=torch.tensor([])
    for i in range(gt.shape[0]):

      IoUs=bops.box_iou(gt[i].reshape(1,4), coords)
      IoUs = IoUs.reshape(1000)
      bbox_idx_to_keep = torch.cat((bbox_idx_to_keep, bbox_idx[IoUs >= thres_IoU]),dim=0)

    X_source = torch.cat((X_source,feat[torch.unique(bbox_idx_to_keep).long()]), dim=0)

In [ ]:
X_source.shape

In [ ]:
#torch.save(X_source, '/content/gdrive/MyDrive/IFT-Projets/X_source_05.pt')

In [ ]:
# center data
scaler = StandardScaler()
X_source_scaled = scaler.fit_transform(X_source)

In [ ]:
# Apply PCA, keep only the first 100 components which gives the Projected source matrix

pca = PCA(n_components=100)
pca.fit(X_source_scaled)

X_source_proj = pca.components_
X_source_proj = torch.from_numpy(X_source_proj)


In [ ]:
X_source_proj.shape

In [ ]:
plt.plot(pca.explained_variance_ratio_) 
plt.grid()

In [ ]:
#torch.save(X_source_proj, '/content/gdrive/MyDrive/IFT-Projets/X_source_proj_05.pt')

### Target data

In [ ]:
# Target data/distribution = trans test set
trans_test_img,_ = get_img_with_bbox(trans_test_ann_path)   # takes about 1min to run on train data
trans_test_data = CustomImageDataset(trans_test_ann_path,img_folder, trans_test_img)
trans_test_dataloader = DataLoader(trans_test_data, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)

 **Construct target matrix:** 
 
We keep output of model.roi_heads.box_head (vector of size 1024) as feature representations of bounding boxes
 extracted by the RPN (region proposal network). For us to stack a box representation to the source matrix, the predicted bbox associated with the feature has to have a confidence score > thres_conf_score (since we don't use target labels we can't use the IoU here).


In [ ]:
thres_conf_score= 0.50 
count=0

X_target=torch.tensor([])

model.eval()

for images, targets in trans_test_dataloader: # trans location valid AND test ?
    images = [image.to(device) for image in images]
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    count+=1

    if count%100==0:
      print(count)

    with torch.no_grad():

        outputs = []
        hook = model.backbone.register_forward_hook(
        lambda self, input, output: outputs.append(output))
        res = model(images)
        hook.remove()

        box_features = model.roi_heads.box_roi_pool(outputs[0], [r['boxes'] for r in res], [i.shape[-2:] for i in images])
        box_features = model.roi_heads.box_head(box_features)

    X_target = torch.cat((X_target,box_features[res[0]['scores']>=thres_conf_score].cpu()), dim=0)


In [ ]:
X_target.shape

In [ ]:
#torch.save(X_target, '/content/gdrive/MyDrive/IFT-Projets/X_target_05.pt')

In [ ]:
# center data
scaler = StandardScaler()
X_target_scaled = scaler.fit_transform(X_target)

In [ ]:
# Apply PCA, keep only the first 100 components which gives the Projected source matrix

pca = PCA(n_components=100)
pca.fit(X_target_scaled)
X_target_proj = pca.components_
X_target_proj = torch.from_numpy(X_target_proj)

In [ ]:
plt.plot(pca.explained_variance_ratio_) # we keep 100 dimensions
plt.grid()

In [ ]:
X_target_proj.shape

In [ ]:
#torch.save(X_target_proj, '/content/gdrive/MyDrive/IFT-Projets/X_target_proj_05.pt')

### Transformation matrix M

In [ ]:
 # 𝑀 is obtained by minimizing the following Bregman matrix divergence (following closed-form solution given in the paper)

In [ ]:
M = torch.matmul(X_source_proj, X_target_proj.T) 

In [ ]:
M.shape

### Project source data into target aligned source subspace

In [ ]:
Xa= torch.matmul(X_source_proj.T,M)

In [ ]:
Xa.shape

In [ ]:
# To project a given feature

# feat(1,1024) x Xa (1024,100)

### Projet target data in target subspace

In [ ]:
# To project a given feature

# feat(1,1024) x X_target_proj.T (1024,100)

### Train adapted model

In [ ]:
# it takes time to generate the following matrices so they are saved 
X_traget_proj = torch.load('your_path')
X_source_proj = torch.load('your_path')

In [ ]:
M = torch.matmul(X_source_proj, X_target_proj.T) # transformation matrix
print(M.shape)

Xa= torch.matmul(X_source_proj.T,M) # target aligned source subspace
print(Xa)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_model_from_pretrained(num_classes)

# move model to the right device
model.to(device)

# load fine-tuned weights
model.load_state_dict(torch.load('your_path/model_25.pt'))


for param in model.parameters(): # to freeze all existing weights

    param.requires_grad = False


# Tricks to include transformation to subspace in the model
model.roi_heads.box_head.add_module('transfo',nn.Linear(in_features=1024, out_features=100, bias=False)) # no bias
model.roi_heads.box_head.transfo.weight = nn.Parameter(Xa, requires_grad = False) # we want to keep these weightd (which are Xa) fixed


model.roi_heads.box_predictor = FastRCNNPredictor(100, 2) # vector are of size 100 after the transformation

# construct an optimizer
# We will only retrain model.roi_heads.box_predictor (2 last layers)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0003, momentum=0.9)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[5,10], gamma=0.1)

In [ ]:
# weights to learn
for i in range(4):

  print(params[i].shape)

In [ ]:
train_valid_img,_= get_img_with_bbox(train_ann_path)  
training_data= CustomImageDataset(train_ann_path,img_folder, train_valid_img)
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True,collate_fn=utils.collate_fn)

In [ ]:
# TRAIN
model.train()

all_train_logs=[]
all_trans_valid_logs=[]
all_cis_valid_logs=[]

num_epochs = 1

for epoch in range(num_epochs):
  # train for one epoch, printing every 10 iterations
  train_logs=train_one_epoch(model, optimizer, train_dataloader, device, epoch, print_freq=100)
  all_train_logs.append(train_logs)
  # update the learning rate
  lr_scheduler.step()
  # evaluate on the test dataset

  for images, targets in trans_valid_dataloader: # can do batch of 10 prob.
    images = [image.to(device) for image in images]
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    with torch.no_grad():
         trans_loss_dict = model(images, targets)
         trans_loss_dict= [{k: loss.to('cpu')} for k, loss in trans_loss_dict.items()]
         all_trans_valid_logs.append(trans_loss_dict)


  for images, targets in cis_valid_dataloader: # can do batch of 10 prob.
    images = [image.to(device) for image in images]
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    with torch.no_grad():
         cis_loss_dict = model(images, targets)
         cis_loss_dict= [{k: loss.to('cpu')} for k, loss in cis_loss_dict.items()]
         all_cis_valid_logs.append(cis_loss_dict)

In [ ]:
# Before testing the model on TRANS TEST, Xa (weights of model.roi_heads.box_head.transfo), has to be replaced by X_traget_proj.T

# Should probably do also for trans valid losses?..

model.roi_heads.box_head.transfo.weight = nn.Parameter(X_traget_proj.T, requires_grad = False) 
